# Vdud Wall

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import tqdm
import random
regex = re.compile("[А-ЯЙ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## 0. Функция, читающая url

In [3]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## 1. Собираем все id постов со страницы

In [4]:
data = readUrl("https://api.vk.com/method/wall.get?owner_id=-145800770&count=100&v=5.95&access_token=")
post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
offset = 101
while offset < 1002:
    data = readUrl("https://api.vk.com/method/wall.get?owner_id=-145800770&offset={}&count=100&v=5.95&access_token=".format(offset))
    new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    post_ids.update(new_post_ids)
    offset += 100

In [5]:
len(post_ids) # 16.06.019 - 362

362

## 2. Собираем все комментарии с капслоком на русском языке по пост-id

In [6]:
# %%time
# с оффсетами
result_entries = []
for id_ in post_ids.keys():
    offset = 1
    while offset < 1002: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id=-145800770&post_id={}&offset={}&count=100&v=6.7&access_token=".format(id_, offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                except TypeError:
                    continue
                    
            comment_ids = [i[3] for i in data_list]
            for com_id in comment_ids:
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id=-145800770&comment_id={}&count=100&v=6.7&access_token=".format(com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                    except TypeError:
                        pass

        except KeyError:
            continue

In [7]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "post_id", "comment_text", "comment_id"])
df_res.count()

from_id         6210
post_id         6210
comment_text    6210
comment_id      6210
dtype: int64

In [8]:
df_res.comment_id.nunique()

6210

In [9]:
df_res.head(100)

,from_id,post_id,comment_text,comment_id
0,15199511.0,174960.0,ХТО ЕТА?,174971
1,153266583.0,174960.0,ПОБЕДА!\n\nhttps://tvrain.ru/news/mvd_prekrati...,175049
2,398674896.0,174960.0,"Очень понравилось, на одном дыхании. Интервью ...",175054
3,26279781.0,174960.0,"ДУДЬ, возьми интервью у своего колеги Голунова...",175060
4,19417313.0,174960.0,"Блин,капец какой умный парень. ""Человеческая ж...",175120
5,146031654.0,174960.0,"Уважаемый, много уважаемый Ю.Дудь,таки здесь б...",175157
6,66484556.0,174960.0,ГОСПОДИ !!ЭТО ЛУЧШЕЕ ИНТЕРВЬЮ СЕЗОНА!!! ОЧЕНЬ ...,175163
7,181037157.0,174960.0,"#конкурс\nМастер 2012 год, режиссера Пола Тома...",175168
8,198783923.0,174960.0,"[id18417696|Юра], ясно все, воспитание под ста...",175097
9,139450192.0,174960.0,Народ вы можете мое письмо Юре отправить?\nЯ в...,175216


## 3. Добавим в датафрейм текст самого поста, чтобы был контекст для определения сентимента

In [10]:
df_res["post_text"] = None

In [11]:
for i in df_res.index:
    df_res["post_text"][i] = post_ids[df_res.post_id[i]]

c:\users\1cc38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
df_res.head()

,from_id,post_id,comment_text,comment_id,post_text
0,15199511.0,174960.0,ХТО ЕТА?,174971,"Балагов - ему 27, а он снова на Каннском фести..."
1,153266583.0,174960.0,ПОБЕДА!\n\nhttps://tvrain.ru/news/mvd_prekrati...,175049,"Балагов - ему 27, а он снова на Каннском фести..."
2,398674896.0,174960.0,"Очень понравилось, на одном дыхании. Интервью ...",175054,"Балагов - ему 27, а он снова на Каннском фести..."
3,26279781.0,174960.0,"ДУДЬ, возьми интервью у своего колеги Голунова...",175060,"Балагов - ему 27, а он снова на Каннском фести..."
4,19417313.0,174960.0,"Блин,капец какой умный парень. ""Человеческая ж...",175120,"Балагов - ему 27, а он снова на Каннском фести..."


## 4. Извлекаем инфу о каждом комментаторе и добавляем в датафрейм. 

In [13]:
df_res["first_name"] = None 
df_res["last_name"] = None 
df_res["sex"] = None
df_res["bdate"] = None 
df_res["city"] = None
df_res.head()

,from_id,post_id,comment_text,comment_id,post_text,first_name,last_name,sex,bdate,city
0,15199511.0,174960.0,ХТО ЕТА?,174971,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None
1,153266583.0,174960.0,ПОБЕДА!\n\nhttps://tvrain.ru/news/mvd_prekrati...,175049,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None
2,398674896.0,174960.0,"Очень понравилось, на одном дыхании. Интервью ...",175054,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None
3,26279781.0,174960.0,"ДУДЬ, возьми интервью у своего колеги Голунова...",175060,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None
4,19417313.0,174960.0,"Блин,капец какой умный парень. ""Человеческая ж...",175120,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None


In [14]:
for i in df_res.index:
    user_id = int(df_res.from_id[i])
    data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=5.95&access_token=".format(user_id))
    try:
        l = list(data["response"])[0]
        l["city"] = l["city"]["title"]
    except KeyError:
        continue
    df_res.loc[i, "first_name"] = l.get("first_name", None)
    df_res.loc[i, "last_name"] = l.get("last_name", None)
    df_res.loc[i, "sex"] = l.get("sex", None)
    df_res.loc[i,"bdate"] = l.get("bdate", None)
    df_res.loc[i,"city"] = l.get("city", None)
    
# работает минут 10

In [15]:
df_res.head()

,from_id,post_id,comment_text,comment_id,post_text,first_name,last_name,sex,bdate,city
0,15199511.0,174960.0,ХТО ЕТА?,174971,"Балагов - ему 27, а он снова на Каннском фести...",None,None,None,None,None
1,153266583.0,174960.0,ПОБЕДА!\n\nhttps://tvrain.ru/news/mvd_prekrati...,175049,"Балагов - ему 27, а он снова на Каннском фести...",Matrena,Sidorova,1,None,Cheboxary
2,398674896.0,174960.0,"Очень понравилось, на одном дыхании. Интервью ...",175054,"Балагов - ему 27, а он снова на Каннском фести...",Varvara,Popova,1,21.6.1980,Moscow
3,26279781.0,174960.0,"ДУДЬ, возьми интервью у своего колеги Голунова...",175060,"Балагов - ему 27, а он снова на Каннском фести...",Alexander,Gorelnikov,2,20.9.1987,Penza
4,19417313.0,174960.0,"Блин,капец какой умный парень. ""Человеческая ж...",175120,"Балагов - ему 27, а он снова на Каннском фести...",Ivan,Blagushin,2,29.6.1989,Stupino


## 5. Оставляем минимальное условие, чтобы для каждого юзера был определен пол

In [16]:
df_test = df_res.dropna(subset=["sex"]) 

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4065 entries, 1 to 6208
Data columns (total 10 columns):
from_id         4065 non-null float64
post_id         4065 non-null float64
comment_text    4065 non-null object
comment_id      4065 non-null int64
post_text       4065 non-null object
first_name      4065 non-null object
last_name       4065 non-null object
sex             4065 non-null object
bdate           3164 non-null object
city            4065 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 349.3+ KB


In [18]:
df_test.sex.value_counts()
# со 100 постов собралось всего 287 комментариев с капслоком 

2    3423
1     642
Name: sex, dtype: int64

In [19]:
df_test.from_id.nunique()

2773

к сожалению, уникальных айди всего 2655 - придется удалить часть строк

In [20]:
df_test.drop_duplicates(subset=["from_id"], keep='first', inplace=True)

c:\users\1cc38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_test = df_test.sort_values(by="post_id")
df_test.count()

from_id         2773
post_id         2773
comment_text    2773
comment_id      2773
post_text       2773
first_name      2773
last_name       2773
sex             2773
bdate           2126
city            2773
dtype: int64

In [22]:
df_test.sex.value_counts()

2    2339
1     434
Name: sex, dtype: int64

## 6. Сохраняем все в файл csv

In [24]:
df_test.to_csv("capitalization-vdud-wall-2.csv", index=False)

In [25]:
df = pd.read_csv("capitalization-vdud-wall.csv")

In [26]:
df.sex.value_counts()

2    2241
1     414
Name: sex, dtype: int64